<a href="https://colab.research.google.com/github/Samgomes2510/dashborad-ecommerce/blob/main/Preparacao_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install pyspark

In [61]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, month
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import ClusteringEvaluator

In [3]:
from google.colab import files
uploaded = files.upload()

Saving videos-comments-tratados.snappy.parquet to videos-comments-tratados.snappy.parquet


In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
spark = SparkSession.builder.getOrCreate()

In [64]:
#1. Ler o arquivo

df_videos = spark.read.option('heaser', 'true').parquet('videos-comments-tratados.snappy.parquet')

In [65]:
from pyspark.sql.functions import month

In [66]:
from pyspark.sql.types import IntegerType # import the IntegerType

df_video = df_video.withColumn('Year', col('Year').cast(IntegerType())) # Now IntegerType will be recognized.

In [67]:
df_video = df_video.withColumn('Year', col('Year').cast(IntegerType()))

In [69]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import month

spark = SparkSession.builder.getOrCreate()

# Ler o parquet corrigindo o typo e conferindo a coluna certa
df_video = spark.read.option('header', 'true').parquet('videos-comments-tratados.snappy.parquet')

# 2 Adicionar a coluna 'Month' usando o nome correto
df_video = df_video.withColumn('Month', month(df_video['Published At']))

In [70]:

df_video = df_video.withColumn('Month', month(df_video['Published At']))

In [71]:
#3. Transformação da coluna 'keyword' para valores numéricos ('Keyword Index')
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(
    inputCol='Keyword',
    outputCol='Keyword Index'
    )

df_video = indexer.fit(df_video).transform(df_video)

In [72]:
# 4. Criar vetor 'Features' com as colunas numéricas
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

# Cast 'Year' column to IntegerType before assembling features
df_video = df_video.withColumn('Year', col('Year').cast(IntegerType()))

assembler = VectorAssembler(
    inputCols=['Likes', 'Views', 'Year', 'Month', 'Keyword Index'],
    outputCol='Features'
)
df_video = assembler.transform(df_video)

In [73]:
# 5. Normalizar o vetor 'Features'
from pyspark.ml.feature import Normalizer # Import the Normalizer class

normalizer = Normalizer(inputCol='Features', outputCol='Features Normal') # Correct the class name to Normalizer
df_video = normalizer.transform(df_video)

In [74]:
# 6. Aplicar PCA para reduzir de 5 dimensões para 1
pca = PCA(k=1, inputCol='Features Normal', outputCol='Features PCA')
pca_model = pca.fit(df_video)
df_video = pca_model.transform(df_video)

In [75]:
# 7. Separar os dados em treino (80%) e teste (20%)
train_data, test_data = df_video.randomSplit([0.8, 0.2], seed=42)

In [76]:
# 8. Treinar modelo de regressão linear para prever 'Comments'
from pyspark.ml.regression import LinearRegression # Import LinearRegression

lr = LinearRegression(featuresCol='Features Normal', labelCol='Comments')
lr_model = lr.fit(train_data)

In [77]:
df_video.show()

+-----------+--------------------+------------+-------+-----+--------+-------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+--------------------+--------------------+
|   Video ID|               Title|Published At|Keyword|Likes|Comments|  Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|Keyword Index|            Features|     Features Normal|        Features PCA|
+-----------+--------------------+------------+-------+-----+--------+-------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+--------------------+--------------------+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|Let's not forget ...|        1|           95|    8|         17.0|[3407.0,135612.0,...|[0.02511243093431...|[0.39522435928489...|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|Here in N

In [78]:
# Avaliar modelo nos dados de teste
test_results = lr_model.evaluate(test_data)
print(f"RMSE (Erro Quadrático Médio): {test_results.rootMeanSquaredError}")
print(f"R² (Coeficiente de Determinação): {test_results.r2}")

RMSE (Erro Quadrático Médio): 43345.23343236093
R² (Coeficiente de Determinação): 0.008252984288786291


In [79]:
# 9. Salvar o dataframe final como parquet
df_video.write.mode('overwrite').parquet('drive/MyDrive/Colab Notebooks/spar/videos-preparados-parquet')